In [72]:
import pandas as pd
import budget
import settings
import digest


In [73]:
a = pd.read_csv("transactions/20239231718.csv")
digest.format_transactions(a)
b = pd.read_csv("transactions/20239301915.csv")
digest.format_transactions(b)

c = pd.concat([a,b]).drop_duplicates(keep=False)
changed = pd.merge(a,c, how="inner")

In [74]:
newer = pd.merge(b,c, how="inner")

In [75]:
a.dtypes

Date                    datetime64[ns]
Description                     object
Original Description            object
Amount                         float64
Transaction Type                object
Category                        object
Account Name                    object
Labels                         float64
Notes                          float64
dtype: object

In [94]:
import textdistance
def no_negs(i):
    if i < 0:
        return 100000
    else:
        return i

def number_similarity(a, b):
    if a-b > abs(a+b):
        return 1
    else:
        a = abs(a)
        b = abs(b)
        return abs(a-b) / (a+b)

def find_closest_debug(row, newer):
    newer["similarity1"] = newer["Date"].apply(lambda x : no_negs((x - row["Date"]).days)/10)
    newer["similarity2"] = newer["Amount"].apply(lambda x : number_similarity(x, row["Amount"]))
    newer["similarity3"] = newer["Description"].apply(lambda x : 1 - textdistance.jaccard(str(x), str(row["Description"])))
    newer["similarity4"] = newer["Account Name"].apply(lambda x : 1 - textdistance.jaccard(str(x), str(row["Account Name"])))
    newer["sim"] = newer["similarity1"] + newer["similarity2"] + newer["similarity3"] + newer["similarity4"]

def find_closest(row, newer):
    a = newer["Date"].apply(lambda x : no_negs((x - row["Date"]).days)/10)
    b = newer["Amount"].apply(lambda x : number_similarity(x, row["Amount"]))
    c = newer["Description"].apply(lambda x : 1 - textdistance.jaccard(str(x), str(row["Description"])))
    d = newer["Account Name"].apply(lambda x : 1 - textdistance.jaccard(str(x), str(row["Account Name"])))
    res = (a+b+c+d)
    return res.idxmin()



In [95]:
changed["ndx"] = changed.apply(lambda row : find_closest(row,newer), axis=1)
i = 6
print(changed.iloc[i], newer.iloc[changed.iloc[i]["ndx"]])

Date                       2023-09-22 00:00:00
Description             ABERCROMBIE & FITCH #0
Original Description    ABERCROMBIE & FITCH #0
Amount                                  -71.55
Transaction Type                        credit
Category                              Clothing
Account Name                Lauren Discover IT
Labels                                     NaN
Notes                                      NaN
ndx                                         33
Name: 6, dtype: object Date                                       2023-09-22 00:00:00
Description             ABERCROMBIE & FITCH #056 WASHINGTON DC
Original Description    ABERCROMBIE & FITCH #056 WASHINGTON DC
Amount                                                  -71.55
Transaction Type                                        credit
Category                                              Clothing
Account Name                                Lauren Discover IT
Labels                                                     NaN
Not

In [96]:
# 6
find_closest_debug(changed.iloc[6], newer)
print(changed.iloc[6])
newer.nsmallest(5, "sim")

Date                       2023-09-22 00:00:00
Description             ABERCROMBIE & FITCH #0
Original Description    ABERCROMBIE & FITCH #0
Amount                                  -71.55
Transaction Type                        credit
Category                              Clothing
Account Name                Lauren Discover IT
Labels                                     NaN
Notes                                      NaN
ndx                                         33
Name: 6, dtype: object


,Date,Description,Original Description,Amount,Transaction Type,Category,Account Name,Labels,Notes,similarity1,similarity2,similarity3,similarity4,sim
33,2023-09-22,ABERCROMBIE & FITCH #056 WASHINGTON DC,ABERCROMBIE & FITCH #056 WASHINGTON DC,-71.55,credit,Clothing,Lauren Discover IT,NaN,NaN,0.0,0.0,0.421053,0.0,0.421053
29,2023-09-22,ABERCROMBIE & FITCH #056 WASHINGTON DC,ABERCROMBIE & FITCH #056 WASHINGTON DC,68.24,debit,Clothing,Lauren Discover IT,NaN,NaN,0.0,1.0,0.421053,0.0,1.421053
32,2023-09-22,FREE PEOPLE #1830 WASHINGTON DC,FREE PEOPLE #1830 WASHINGTON DC,156.88,debit,Clothing,Lauren Discover IT,NaN,NaN,0.0,1.0,0.605263,0.0,1.605263
30,2023-09-22,MADEWELL #243 WASHINGTON DC,MADEWELL #243 WASHINGTON DC,170.12,debit,Clothing,Lauren Discover IT,NaN,NaN,0.0,1.0,0.638889,0.0,1.638889
31,2023-09-22,SMARTRIP WASHINGTON DC WASHINGTON DC\nAPPLE PA...,SMARTRIP WASHINGTON DC WASHINGTON DC\nAPPLE PA...,12.00,debit,Electronics & Software,Lauren Discover IT,NaN,NaN,0.0,1.0,0.742424,0.0,1.742424
